In [2]:
import pandas as pd
import random
import numpy as np
df = pd.read_csv("all_history.csv")
df = df.set_index("Date")
df.head(5)

,A,AA,AAAP,AABB,AAC,AAL,AAMC,AAME,AAN,AAOI,...,ZSTN,ZTR,ZTS,ZULU,ZUMZ,ZVLO,ZVTK,ZWBC,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
01/03/2000,43.76,70.69,24.50,77.49,9.87,18.19,9.26,2.09,24.51,9.96,...,7.52,2.46,29.05,0.06,12.44,26.75,3750000.0,12.75,16.25,1.28
01/04/2000,40.42,71.02,25.00,77.49,9.96,19.33,8.65,2.15,20.62,10.10,...,7.20,2.48,29.06,0.06,12.26,25.00,10000000.0,12.75,19.32,1.31
01/05/2000,37.91,75.11,25.26,77.49,9.78,19.05,8.26,2.20,17.75,10.00,...,7.50,2.49,29.08,0.05,12.05,27.50,7500000.0,30.00,24.54,1.32
01/06/2000,36.46,74.13,25.02,77.49,9.83,19.81,8.39,2.09,17.02,10.00,...,7.55,2.51,29.07,0.05,12.12,25.50,4450000.0,25.00,35.14,1.31
01/07/2000,39.50,73.91,24.62,77.49,9.90,20.27,8.96,2.15,16.88,9.97,...,7.31,2.56,29.98,0.05,11.75,27.34,2750000.0,16.00,29.73,1.29


In [241]:
def get_history(begin_date, end_date, stock_index):
    # Gets the rows between begin_date and end_date for the stock at df[stock_index]
    sub_df = df.iloc[begin_date:end_date, stock_index]
    
    print(sub_df)
    return sub_df
        

ticker_df = get_history(0, 10, 1)



Date
01/03/2000    70.69
01/04/2000    71.02
01/05/2000    75.11
01/06/2000    74.13
01/07/2000    73.91
01/10/2000    73.69
01/11/2000    73.26
01/12/2000    72.60
01/13/2000    71.29
01/14/2000    69.87
Name: AA, dtype: float64


In [242]:
def randomize_ticker():
    ticker_index = random.randint(0, len(df.columns))
    sub_df = df.iloc[:, ticker_index]
    return sub_df.name.upper()
    

def randomize_start(period_length):
    """Takes a length parameter that will be added to a random start date and represent the period analyzed"""
    rand_start = random.randint(0, (len(df) - period_length))
    
    return rand_start
    

#   Trim null-valued prices from the df
#     prices = df[ticker.upper()]
#     prices = prices.dropna()
    
#   Trim possible dates so that rand_start + period_length will not throw an out-of-bounds exception
    
    
    
def create_portfolio(period_length, num_tickers):
#   df to hold the data from each ticker
    prices_df = pd.DataFrame({})

    start_index = randomize_start(period_length)
    
    i = 0
    while i < num_tickers:
#       Run the randomizer and trim values
        ticker = randomize_ticker()
        ticker_index = df.columns.get_loc(ticker)
        ticker_df = df.iloc[start_index : start_index + period_length, [ticker_index]]

#       Clean null values and randomize again if necessary
        ticker_df = ticker_df.dropna()
        if (len(ticker_df) < period_length):
            # Break and decrement i if there are null values in the date range
            print("\nSeries contains null values.  Choosing a different ticker...")
            i -= 1

        else:
            # Add the data as intended.
            if prices_df.empty:
                prices_df = ticker_df
            else:
                prices_df = pd.concat([prices_df, ticker_df], axis=1)
#                   prices_df = prices_df.merge(prices_df, ticker_df)
#       
        i += 1
    
    
    print("\n\nFinal prices_df: ")
    print(prices_df.head(10))
    
    return prices_df
    
    
# test_ticker = randomize_ticker()
# print(test_ticker)

# randomize_start(10)


In [243]:
# Assign random weights
def assignWeights(df, method="Random"):
    """Takes a df and assigns weights (defaults to random, but can be either Random or Equal.)
    Returns a weights df with columns corresponding to the columns in the df arg.
    """
    # Dict to store weights as values to column name keys
    df_weights = {}

    # Since the df.columns length will at this point include % change values, weights length should be halved.
    weight_array_length = int(len(df.columns) / 2)
    
    weights = []
    for weight in range(weight_array_length):
        weights.append(random.randint(0, 10000))
        
    new_weights = []
    i = 0
    for weight in weights:
        new_weights.append(round(weights[i] / sum(weights), 2))
        i += 1
        
    print("sum:")
    print(sum(new_weights))
    
    # Check whether there are any errors and make the sum = 100
    if (sum(new_weights) < 1):
        # choose a random index
        randIndex = random.randint(0, len(new_weights))
        print(new_weights[randIndex])
        new_weights[randIndex] += (1 - sum(new_weights))
        print(new_weights)
    
    # Only assign random weights to the columns that don't contain %
    if (method == "Random"):
        for item in df.columns:
            if "%" not in item:         
                # Add to the dictionary
                df_weights[str(item)] = new_weights[df.columns.get_loc(item)]
            
    elif (method == "Equal"):
        # Assign equally
        df[str(item)+" Weight"] = 1 / (len(df.columns) / 2)
        
    else:
        print("Choose weighting method: Random or Equal.")
    
    print(df_weights)
    return df_weights


In [251]:
sample_portfolio_1 = create_portfolio(6, 3)
sample_portfolio_1.head()

for item in sample_portfolio_1.columns:
#     First, load with values
    sample_portfolio_1[str(item)+" % chg"] = sample_portfolio_1[item].pct_change()


def get_portfolio_stats(portfolio_df, portfolio_weights):
    """Takes a portfolio price history df and its corresponding weights dict from an assign_weights function,
    and returns the weighted HPR and annualized return for the portfolio"""
    
    hpr_items = []
    stdev_items = []
    
    # Reset index to make using loc practical
    portfolio_df = portfolio_df.reset_index()
    non_date_columns = portfolio_df.iloc[:, 1:]
    
    # Compute total cumulative % change for each asset
    for item in non_date_columns.columns:
        if ("%" not in item) & ("/" not in item):
            
            beginning_price = non_date_columns.loc[0 , item]
            # print("Beginning price:")
            # print(beginning_price)
            
            ending_price = non_date_columns.loc[non_date_columns.index[-1], item]
            # print("Ending price:")
            # print(ending_price)
            
            item_hpr = (ending_price - beginning_price) / beginning_price
            # print("HPR for "+str(item)+":")
            # print(item_hpr)
            
            hpr_items.append(item_hpr)
            
        elif ("%" in item):
        # Compute standard deviation of daily % changes for each asset
            col_std = non_date_columns[item].std()
            stdev_items.append(col_std)
            
        
        # Multiply by weights...
        # First for HPR array
        weighted_hpr_items = []
        i = 0
        for item in hpr_items:
            weighted_hpr_items.append(hpr_items[i] * portfolio_weights[non_date_columns.columns[i]])
            i += 1
        # Then for the stdev array
        weighted_stdev_items = []
        i = 0
        for item in stdev_items:
            weighted_stdev_items.append(stdev_items[i] * portfolio_weights[non_date_columns.columns[i]])
            i += 1
            
    print(weighted_hpr_items)
    portfolio_hpr = sum(weighted_hpr_items)
    print(portfolio_hpr)
    
    print(weighted_stdev_items)
    portfolio_stdev = sum(weighted_stdev_items)
    print(portfolio_stdev)
    
    # Adjust by the period analyzed...
    
    # Compute the percentage of a trading year this period represents
    trading_days = 252
    df_days = len(non_date_columns.iloc[:, 0])
    print("trading days in period analyzed: "+str(df_days))
    pct_of_year = df_days / trading_days
    print("% of trading year: "+str(pct_of_year))
    
    # Extrapolate for annualized return using appropriate formula
    annualized_hpr = (1 + portfolio_hpr) ** (1 / pct_of_year) - 1
    print("annualized return: "+str(annualized_hpr))
    
    # Apply a similar process to the standard deviation using the sqrt of period analyzed
    annualized_volatility = portfolio_stdev * np.sqrt(df_days)
    print("annualized volatility: "+str(annualized_volatility))
    
    # Now compute annualized risk-adjusted return
    risk_adj_return = annualized_hpr / annualized_volatility
    print("Risk-Adjusted Return: "+str(risk_adj_return))

    # Convert all of this information on the portfolio to its own dataframe
    portfolio_statistics = {
        "num_days": [df_days],
        "period_start": [portfolio_df.loc[0, "Date"]],
        "period_end": [portfolio_df.loc[portfolio_df.index[-1], "Date"]],
        "num_stocks": [int(len(non_date_columns.columns) / 2)],
        "hpr": [portfolio_hpr],
        "annualized_return": [annualized_hpr],
        "stdev": [portfolio_stdev],
        "annualized_stdev": [annualized_volatility],
        "risk_adj_return": [risk_adj_return]
    }
    portfolio_statistics_df = pd.DataFrame(portfolio_statistics)
    
    # Output
    return portfolio_statistics_df
    
    
# Testbed
get_portfolio_stats(sample_portfolio_1, assignWeights(sample_portfolio_1, "Random")).head()
# sample_portfolio_1.head(50)






Final prices_df: 
            LMFA  UFMG  FRTG
Date                        
07/28/2006  0.84  7.11  0.81
07/31/2006  0.86  7.11  0.81
08/01/2006  0.85  7.11  0.81
08/02/2006  0.85  7.11  0.81
08/03/2006  0.92  7.11  0.81
08/04/2006  0.92  7.11  0.81
sum:
0.99
0.35
[0.36, 0.5, 0.14]
{'LMFA': 0.36, 'UFMG': 0.5, 'FRTG': 0.14}
[0.034285714285714315, 0.0, 0.0]
0.034285714285714315
[0.013585829598730424, 0.0, 0.0]
0.013585829598730424
trading days in period analyzed: 6
% of trading year: 0.023809523809523808
annualized return: 3.1200463297249863
annualized volatility: 0.03327835024929027
Risk-Adjusted Return: 93.75603977818966


,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
0,6,07/28/2006,08/04/2006,3,0.034286,3.120046,0.013586,0.033278,93.75604


In [16]:
# We will store the period, start date, and portfolios in a class file.  We can then have
# an array of Portfolios to compare.

class Portfolio:
    # instance variables/attributes...
    size = 10 # The initial size or num_stocks in the portfolio
    n_periods = 10 # The number of trading days the portfolio analysis takes place over
    start_date_index = 0
    
    weights = pd.DataFrame({}) # Weights that will be assigned to each of the assets in the portfolio
    
    stdev = 0
    hpr = 0
    hpr_annualized = 0
    risk_adj_return = 0
    alpha = 0
    
    history_df = pd.DataFrame({})
    analysis_df = pd.DataFrame({})
    
    # Constructor
    def __init__(self, size, n_periods):
        
        self.size = size
        self.n_periods = n_periods
        
        # Randomize starting period within csv
        self.randomize_start()
        
        # Generate history_df
        self.create_portfolio()
        
        # Assign weights
        self.assignWeights()
        
        print("Initialized new Portfolio instance")
        
    
    # Methods...
    
    # NOTE: Both randomize_ricker and randomize_start need access to the massive CSV file and currently take a pre-made df
    # not as an argument, but as a variable inside them.  This will need to be changed in whatever file I move this to.
    def randomize_ticker(self):
        ticker_index = random.randint(0, len(df.columns))
        sub_df = df.iloc[:, ticker_index]
        return sub_df.name.upper()
    

    def randomize_start(self):
        """Takes a length parameter that will be added to a random start date and represent the period analyzed"""
        period_length = self.n_periods
        # NOTE: This uses the length of df, which does not exist in the class.  Might be better to define the read_csv here in the class file.
        # Perhaps pass it as an argument in randomize_start and have that param point to the read_csv
        rand_start = random.randint(0, (len(df) - period_length))
        
        self.start_date_index = rand_start
        return self.start_date_index
    
    
    def create_portfolio(self):
    #   df to hold the data from each ticker
        prices_df = pd.DataFrame({})

        start_index = self.start_date_index
        
        i = 0
        while i < self.size:
    #       Run the randomizer and trim values
            ticker = self.randomize_ticker()
            ticker_index = df.columns.get_loc(ticker)
            ticker_df = df.iloc[start_index : start_index + self.n_periods, [ticker_index]]

    #       Clean null values and randomize again if necessary
            ticker_df = ticker_df.dropna()
            if (len(ticker_df) < self.n_periods):
                # Break and decrement i if there are null values in the date range
                # print("\nSeries contains null values.  Choosing a different ticker...")
                i -= 1

            else:
                # Add the data as intended.
                if prices_df.empty:
                    prices_df = ticker_df
                else:
                    prices_df = pd.concat([prices_df, ticker_df], axis=1)
    #                   prices_df = prices_df.merge(prices_df, ticker_df)
    #       
            i += 1
        
        # Load with the percentage change columns
        for item in prices_df.columns:
    #       First, load with values
            prices_df[str(item)+" % chg"] = prices_df[item].pct_change()
        
        # print("\n\nFinal prices_df: ")
        print(prices_df.head(3))
        
    
        self.history_df = prices_df
        return self.history_df
    
    
#   Assign weights
    def assignWeights(self, method="Random"):
        """Takes a df and assigns weights (defaults to random, but can be either Random or Equal.)
        Returns a weights df with columns corresponding to the columns in the df arg.
        """
        # Dict to store weights as values to column name keys
        df_weights = {}

        # Since the df.columns length will at this point include % change values, weights length should be halved.
        weight_array_length = int(len(self.history_df.columns) / 2)
        
        weights = []
        for weight in range(weight_array_length):
            weights.append(random.randint(0, 10000))
            
        new_weights = []
        i = 0
        for weight in weights:
            new_weights.append(round(weights[i] / sum(weights), 2))
            i += 1
            
        print("weights array:")
        print(str(new_weights))
        
        # Check whether there are any errors and make the sum = 100
        if (sum(new_weights) < 1):
            # choose a random index
            randIndex = random.randint(0, len(new_weights))
            print(new_weights[randIndex])
            new_weights[randIndex] += (1 - sum(new_weights))
            print(new_weights)
        
        # Only assign random weights to the columns that don't contain %
        if (method == "Random"):
            print("self.history_df.columns: "+str(self.history_df.columns))
            for item in self.history_df.columns:
                print("\nitem: "+str(item))
                if "%" not in item:         
                    print("% not in item.")
                    # Add to the dictionary
                    print("df_weights[item] now being set to new_weights[self.history_df.get_loc(item)]: ")
                    print(str(new_weights[self.history_df.columns.get_loc(item)]))
                    df_weights[str(item)] = new_weights[self.history_df.columns.get_loc(item)]
                
        elif (method == "Equal"):
            # Assign equally
            df[str(item)+" Weight"] = 1 / (len(df.columns) / 2)
            
        else:
            print("Choose weighting method: Random or Equal.")
            
        self.weights = df_weights
        print(self.weights)
        return self.weights
        

    # Compute portfolio statustics
    def get_portfolio_stats(self):
        """Takes a portfolio price history df and its corresponding weights dict from an assign_weights function,
        and returns the weighted HPR and annualized return for the portfolio"""
        
        hpr_items = []
        stdev_items = []
        
        # Reset index to make using loc practical
        self.history_df = self.history_df.reset_index()
        non_date_columns = self.history_df.iloc[:, 1:]
        
        # Compute total cumulative % change for each asset
        for item in non_date_columns.columns:
            print("\nitem: "+str(item))
            if ("%" not in item) & ("/" not in item):
                print("% and / not in item.")
                
                beginning_price = non_date_columns.loc[0 , item]
                # print("Beginning price:")
                # print(beginning_price)
                
                ending_price = non_date_columns.loc[non_date_columns.index[-1], item]
                # print("Ending price:")
                # print(ending_price)
                
                item_hpr = (ending_price - beginning_price) / beginning_price
                # print("HPR for "+str(item)+":")
                # print(item_hpr)
                
                hpr_items.append(item_hpr)
                
            elif ("%" in item):
            # Compute standard deviation of daily % changes for each asset
                col_std = non_date_columns[item].std()
                stdev_items.append(col_std)
                
            
            # Multiply by weights...
            # First for HPR array
            weighted_hpr_items = []
            i = 0
            for item in hpr_items:
                weighted_hpr_items.append(hpr_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
            # Then for the stdev array
            weighted_stdev_items = []
            i = 0
            for item in stdev_items:
                weighted_stdev_items.append(stdev_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
                
        print(weighted_hpr_items)
        portfolio_hpr = sum(weighted_hpr_items)
        print(portfolio_hpr)
        
        print(weighted_stdev_items)
        portfolio_stdev = sum(weighted_stdev_items)
        print(portfolio_stdev)
        
        # Adjust by the period analyzed...
        
        # Compute the percentage of a trading year this period represents
        trading_days = 252
        df_days = len(non_date_columns.iloc[:, 0])
        print("trading days in period analyzed: "+str(df_days))
        pct_of_year = df_days / trading_days
        print("% of trading year: "+str(pct_of_year))
        
        # Extrapolate for annualized return using appropriate formula
        annualized_hpr = (1 + portfolio_hpr) ** (1 / pct_of_year) - 1
        print("annualized return: "+str(annualized_hpr))
        
        # Apply a similar process to the standard deviation using the sqrt of period analyzed
        annualized_volatility = portfolio_stdev * np.sqrt(df_days)
        print("annualized volatility: "+str(annualized_volatility))
        
        # Now compute annualized risk-adjusted return
        risk_adj_return = annualized_hpr / annualized_volatility
        print("Risk-Adjusted Return: "+str(risk_adj_return))

        # Convert all of this information on the portfolio to its own dataframe
        portfolio_statistics = {
            "num_days": [df_days],
            "period_start": [self.history_df.loc[0, "Date"]],
            "period_end": [self.history_df.loc[self.history_df.index[-1], "Date"]],
            "num_stocks": [int(len(non_date_columns.columns) / 2)],
            "hpr": [portfolio_hpr],
            "annualized_return": [annualized_hpr],
            "stdev": [portfolio_stdev],
            "annualized_stdev": [annualized_volatility],
            "risk_adj_return": [risk_adj_return]
        }
        stats_df = pd.DataFrame(portfolio_statistics)
        self.analysis_df = stats_df
        
        # Output
        #   Write analysis metrics to analysis_df
        return self.analysis_df

    #   Compare analysis to benchmark for alpha

    
    
    
# Testbed
portfolio_1 = Portfolio(4, 8)
portfolio_1.get_portfolio_stats()


            PHOS  HOFT    WTS    IMO  PHOS % chg  HOFT % chg  WTS % chg  \
Date                                                                      
06/01/2009   0.0  7.70  18.29  33.72         NaN         NaN        NaN   
06/02/2009   0.0  7.39  18.09  33.65         NaN   -0.040260  -0.010935   
06/03/2009   0.0  7.21  18.33  32.05         NaN   -0.024357   0.013267   

            IMO % chg  
Date                   
06/01/2009        NaN  
06/02/2009  -0.002076  
06/03/2009  -0.047548  
weights array:
[0.46, 0.0, 0.33, 0.21]
self.history_df.columns: Index(['PHOS', 'HOFT', 'WTS', 'IMO', 'PHOS % chg', 'HOFT % chg', 'WTS % chg',
       'IMO % chg'],
      dtype='object')

item: PHOS
% not in item.
df_weights[item] now being set to new_weights[self.history_df.get_loc(item)]: 
0.46

item: HOFT
% not in item.
df_weights[item] now being set to new_weights[self.history_df.get_loc(item)]: 
0.0

item: WTS
% not in item.
df_weights[item] now being set to new_weights[self.history_df.get_loc(it

C:\Users\Jonathan\AppData\Local\Temp\ipykernel_8580\2259050428.py:185: RuntimeWarning: invalid value encountered in double_scalars
  item_hpr = (ending_price - beginning_price) / beginning_price


,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
0,8,06/01/2009,06/10/2009,4,NaN,NaN,NaN,NaN,NaN
